**Импортируем необходимые библиотеки**

In [85]:
import cv2 as cv
import sys
import numpy as np
import math as mt
from matplotlib import pyplot as plt
import glob
import os 
np.random.seed(0)

* *Функция для сохранения картинки*

In [86]:
def SaveImage(img,name,dst=''):
    cv.imwrite(dst+name,img)

* *Функции подавлнеия шума: Билетиральный фильтр, Эрозия*

In [87]:
def ProcessBileteral(image,name):
    bileteral=cv.bilateralFilter(image, 15 ,  75 , 75)
    SaveImage(bileteral,name,'method/bileteral/denoising/')
    return bileteral

def ProcessErode(image,name,kernel):
    erode=cv.erode(image,kernel)
    SaveImage(image,name,'method/erosion/denoising/')
    return erode

In [88]:
def CalculateMask(image,name,dst):

    hsv_image=cv.cvtColor(image,cv.COLOR_BGR2HSV)# Конвертируем изображение в HSV формат

    healty_part=cv.inRange(hsv_image,(36,10,25),(86,255,255)) # Выделяем здоровую область
    background=cv.inRange(hsv_image,(90,5,25),(240,255,255))+cv.inRange(hsv_image,(0,5,31),(2,255,255)) # Выделяем фон
    shadows=cv.inRange(hsv_image,(0,0,0),(240,255,25)) # Выделяем тень

    
    #Делаем случайную маску с шансами p1 и p2  для того чтобы доавлять маркеры случайно
    p1,p2=1,1
    healty_randomized_mask=np.random.choice(a=[True, False], size=(image.shape[0],image.shape[1]), p=[p1, 1-p1])  
    background_randomized_mask=np.random.choice(a=[True, False], size=(image.shape[0],image.shape[1]), p=[p2, 1-p2]) 

    #healty_mask=healty_part and healty_randomized_mask
    healty_mask=np.all([healty_part>0,healty_randomized_mask==True],axis=0)

    #background_mask=(shadows or background) and background_randomized_mask
    background_mask=np.any([background>0,shadows>0],axis=0)
    background_mask=np.all([background_mask,background_randomized_mask==True],axis=0)

    #Рисуем картинку с маркерами
    markers_visualization=np.zeros_like(image,np.uint8)
    markers_visualization[healty_mask==True]=(0,230,100)
    markers_visualization[background_mask==True]=(0,100,230)
    
    SaveImage(np.concatenate([markers_visualization,image],axis=1),name,dst+'markers/')
    

    #Делаем маркеры
    markers=np.zeros((image.shape[0],image.shape[1]),dtype='int32')

    markers[healty_mask==True]=255
    markers[background_mask==True]=1

    #Запускаем алгоритм
    labels =cv.watershed(hsv_image,markers)
    print( np.unique(labels))
    colors=[(125,0,255),(0,0,0),(125,255,0),(0,255,255)]
   
    result=np.zeros_like(image,np.uint8)
    i=0
    for label in np.unique(labels): 
        result[labels == label] = colors[i]
        i+=1

    watershed_img= np.copy(result)
    SaveImage(watershed_img,name,dst+'watershed/')

    #Находим больную часть
    ill_part=labels-healty_part
    result[ill_part>1]=colors[3]
    SaveImage(result,name,dst+'recognizing/')

    #Сохраняем и показываем все этапы картинки
    conc=np.concatenate([image,markers_visualization,watershed_img,result],axis=1)
    SaveImage(conc,name,dst+'summary/')
    cv.imshow('Display window',conc)
    cv.waitKey(0)





* *Полная обработка всех изображений для разных методов обработки шума*

In [89]:
def ProcessImagesBileteral(images_paths):
    for image_path in images_paths:
        image=cv.imread(image_path)
        name=image_path[image_path.rfind('\\')+1:]
        bileteral=ProcessBileteral(image,name)
        CalculateMask(bileteral,name,'method/bileteral/')

def ProcessImagesErode(images_paths):
    for image_path in images_paths:
        image=cv.imread(image_path)
        name=image_path[image_path.rfind('\\')+1:]
        kernel=cv.getStructuringElement(cv.MORPH_ELLIPSE,(7,7))
        erode=ProcessErode(image,name,kernel)
        CalculateMask(erode,name,'method/erosion/')

* *Получение путей всех картинок и запуск их обработки*

In [90]:
print(os.getcwd()) 
images_paths=[path for path in glob.glob('test/*.jpg')]
ProcessImagesBileteral(images_paths)
cv.destroyAllWindows()

e:\PROGRAMMING\Python\Image Processing
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
[ -1   1 255]
